1. Load the dataset and Create an NLP Model

In [1]:
# import the required libraries
import spacy
import random

 create a model if there is no existing model otherwise load the existing model

In [14]:
# Initial parameters
model_file = None # set esisting model name other wise set it to None
iterations= 50 # Training data
TRAINING_DATA = [('what is the price of McVeggie?', {'entities': [(21, 29, 'FoodProduct')]}),
                 ('what is the price of McEgg?', {'entities': [(21, 26, 'FoodProduct')]}),
                 ('what is the price of McChicken?', {'entities': [(21, 30, 'FoodProduct')]}),
                 ('what is the price of McSpicy Paneer?', {'entities': [(21, 35, 'FoodProduct')]}),
                 ('what is the price of McSpicy Chicken?', {'entities': [(21, 36, 'FoodProduct')]})] 
# Testing sample data       
test_sample='what is the price of McAloo?'
# Create NLP model
if model_file is not None:
    " "
    nlp = spacy.load(model_file)  
    print("Load Existing NER Model ", model_file)
else:
    nlp = spacy.blank('en')  
    print("Created blank NLP model")

Created blank NLP model


Create an NLP Pipeline
First check for exisitng pipelines, if not create new one

In [15]:
# Create NLP Pipeline
if 'ner' not in nlp.pipe_names:
    ner_pipe = 'ner'
    nlp.add_pipe(ner_pipe, last=True)
    ner_pipe = nlp.get_pipe('ner')
else:
    ner_pipe = nlp.get_pipe('ner')

Add Entities Labels

add entities' labels to the pipeline. First, iterate the training dataset and then add each entity to the model

In [16]:
# Add entities labels to the ner pipeline
for text, annotations in TRAINING_DATA:
    for entity in annotations.get('entities'):
        ner_pipe.add_label(entity[2])

Train NER model

First, disable all other pipelines and then go only NER training. In NER training, create an optimizer. after that, update nlp model based on text and annotations in the training dataset. This process continues to a defined number of iterations.

In [17]:
from spacy.training.example import Example

In [18]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']# train NER Model

with nlp.disable_pipes(*other_pipes):  # only train NER
    nlp.begin_training()
    for itn in range(iterations):
        print("Iteration Number:" + str(itn))
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in TRAINING_DATA:
            doc = nlp.make_doc(text)
            print(doc,annotations)
            example = Example.from_dict(doc, annotations)
            nlp.update([example],drop=0.2,sgd=optimizer,losses=losses)
        print("Loss:",losses)

Iteration Number:0
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
Loss: {'ner': 27.445197105407715}
Iteration Number:1
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
Loss: {'ner': 14.661988038569689}
Iteration Number:2
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProdu

what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
Loss: {'ner': 4.429239125574989e-07}
Iteration Number:20
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
Loss: {'ner': 2.021836875626671e-08}
Iteration Number:21
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
Loss: {'ner': 9.495222592839442e-08}
Iteration Number:22
what is the price of

what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
Loss: {'ner': 4.0672020555790874e-08}
Iteration Number:40
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
Loss: {'ner': 6.071501396638602e-08}
Iteration Number:41
what is the price of McSpicy Chicken? {'entities': [(21, 36, 'FoodProduct')]}
what is the price of McChicken? {'entities': [(21, 30, 'FoodProduct')]}
what is the price of McVeggie? {'entities': [(21, 29, 'FoodProduct')]}
what is the price of McEgg? {'entities': [(21, 26, 'FoodProduct')]}
what is the price of McSpicy Paneer? {'entities': [(21, 35, 'FoodProduct')]}
Loss: {

In [19]:
# save model
#nlp.to_disk(model_file)

In [20]:
# test model
test_document = nlp(test_sample)

for ent in test_document.ents:
    print(ent)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

McAloo
McAloo 21 27 FoodProduct
